In [ ]:
# !pip install geocoder
# !pip install meteostat
# !pip install geopandas
# !pip install osmnx

In [ ]:
import os
import requests
from datetime import datetime, timedelta, date
import pandas as pd
from meteostat import Point, Daily
from google.colab import files
import geocoder
from geopy.geocoders import Nominatim
from tqdm import tqdm
import geopandas as gpd
import osmnx as ox
from shapely.geometry import Point

***Input*** ***1: Crop Information: Names & Locations***

In [ ]:
df = pd.read_excel("harvest_locations.xlsx", skiprows=1)

# state_location = df["Kt."]
# origin_location = df["Standort"]
crop_names = df.columns[7:].tolist()

mapping = {
    'Brotweizen': 'Wheat',
    'Umstellmahlweizen': 'Wheat',
    'Futterweizen': 'Wheat',
    'Futtergerste': 'Barley',
    'Körnermais': 'Corn',
    'Sojabohnen Futter': 'Soybean',
    'Sojabohnen "Tofu"': 'Soybean',
    'Raps Kl.': 'Rapeseed',
    'Raps HOLL': 'Rapeseed',
    'Sonnenblumen lino': 'Sunflowerseed',
    'Sonnenblumen HO': 'Sunflowerseed'
}


swiss_crop = [crop for crop in crop_names if crop in mapping]

locations_dict = {}

for crop in swiss_crop:
    category = mapping[crop]  # 找到大类
    crop_locations = df.loc[df[crop] == "x", "Standort"].tolist()

    if category not in locations_dict:
        locations_dict[category] = crop_locations
    else:
        locations_dict[category] = list(set(locations_dict[category] + crop_locations))


for k, v in locations_dict.items():
    print(f"{k}: {v}")
    print(f"{k}: {len(v)} locations")



Wheat: ['1254 Jussy', '4573 Lohn-Ammannsegg', '7302 Landquart', '1784 Courtepin', '2800 Delémont', '1072 Forel VD', '3254 Messen', '2900 Porrentruy', '8308 Illnau', '1462 Yvonand', '1868 Collombey-le-', '5742 Kölliken', '4950 Huttwil', '4460 Gelterkinden', '1580 Avenches', '8194 Hüntwangen', '1350 Orbe', '3054 Schüpfen', '3076 Worb', '3114 Wichtrach', '8155 Niederhasli', '3423 Ersigen', '8460 Marthalen', '3292 Busswil b. Büren', '1487 Cugy', '8903 Birmensdorf', '1610 Oron-la-Ville', '3661 Uetendorf', '4800 Zofingen', '3360 Herzogenbuchsee', '4242 Laufen', '9494 Schaan (FL)', '1242 Satigny', '3210 Kerzers', '3415 Hasle-Rüegsau', '2733 Pontenet', '8162 Steinmaur', '2053 Cernier', '8932 Mettmenstetten', '4538 Oberbipp', '5610 Wohlen', '3186 Düdingen', '8207 Schaffhausen', '2942 Alle', '3421 Lyssach', '6210 Sursee', '1038 Bercher', '4917 Melchnau', '8560 Märstetten', '3177 Laupen', '5074 Eiken', '3940 Steg', '1305 Penthalaz']
Wheat: 53 locations
Barley: ['5742 Kölliken', '5074 Eiken', '561

In [ ]:
geolocator = Nominatim(user_agent="geoapi", timeout=10)

cache = {}

def clean_location(address):
    address = address.replace("b.", "bei")
    if "(FL)" in address:
        address = address.replace("(FL)", "").strip()
        country = "Liechtenstein"
    else:
        country = "Switzerland"
    return address, country

def get_lat_lon_cached(address):
    if address in cache:
        return cache[address]

    try:
        clean_addr, country = clean_location(address)
        location = geolocator.geocode(f"{clean_addr}, {country}")
        if location:
            result = (location.latitude, location.longitude)
        else:
            result = (None, None)
    except:
        result = (None, None)

    manual_fix = {
        "3292 Busswil bei Büren": (47.133, 7.246),
        "9494 Schaan": (47.166, 9.516)
    }
    if result == (None, None) and clean_addr in manual_fix:
        result = manual_fix[clean_addr]

    cache[address] = result
    return result

category_coords = {}
for category, locations in locations_dict.items():
    print(f"Processing {category}...")
    coords_list = []
    for loc in tqdm(locations):
        lat, lon = get_lat_lon_cached(loc)
        coords_list.append([loc, lat, lon])
    category_coords[category] = pd.DataFrame(coords_list, columns=["Standort", "Latitude", "Longitude"])

wheat_loc = category_coords["Wheat"]
barley_loc = category_coords["Barley"]
corn_loc = category_coords["Corn"]
soy_loc = category_coords["Soybean"]
rap_loc = category_coords["Rapeseed"]
sun_loc = category_coords["Sunflowerseed"]

Processing Wheat...


100%|██████████| 53/53 [00:54<00:00,  1.02s/it]


Processing Barley...


100%|██████████| 49/49 [00:00<00:00, 294021.31it/s]


Processing Corn...


...

100%|██████████| 34/34 [00:00<00:00, 187147.42it/s]


Processing Rapeseed...


100%|██████████| 25/25 [00:00<00:00, 193107.92it/s]


Processing Sunflowerseed...


100%|██████████| 22/22 [00:00<00:00, 201473.12it/s]


In [ ]:
print(wheat_loc)
for i in wheat_loc:
  print(i)
  print(type(i))

                 Standort   Latitude  Longitude
0              1254 Jussy  46.232044   6.272693
1    4573 Lohn-Ammannsegg  47.170713   7.528663
2          7302 Landquart  46.962238   9.563587
3          1784 Courtepin  46.865568   7.122968
4           2800 Delémont  47.364306   7.345486
5           1072 Forel VD  46.540140   6.769301
6             3254 Messen  47.099072   7.435817
7         2900 Porrentruy  47.382472   7.098541
8             8308 Illnau  47.409475   8.723691
9            1462 Yvonand  46.797683   6.740887
10     1868 Collombey-le-  46.279782   6.948920
11          5742 Kölliken  47.334149   8.021650
12           4950 Huttwil  47.112213   7.849174
13      4460 Gelterkinden  47.464226   7.855537
14          1580 Avenches  46.880078   7.040362
15        8194 Hüntwangen  47.594975   8.493507
16              1350 Orbe  46.724953   6.532812
17          3054 Schüpfen  47.039854   7.375502
18              3076 Worb  46.930236   7.566273
19         3114 Wichtrach  46.844390   7

***Input 2: Climate Data***

* Location

In [ ]:
latitude = 47.3341488
longitude = 8.0216498

* Historical and future weather forecast

In [ ]:
def get_historical_weather(start_date_str, end_date_str):
    """
    Fetch historical daily weather (ERA5).
    ERA5 typically has ~5 days delay, so yesterday is not available.
    """
    ERA5_URL = "https://archive-api.open-meteo.com/v1/era5"

    # Adjust end_date if too recent
    today = date.today()
    max_available = today - timedelta(days=5)
    end_date = min(datetime.strptime(end_date_str, "%Y-%m-%d").date(), max_available)
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d").date()

    if start_date > end_date:
        raise ValueError("Start date is after available end date.")

    daily_vars = ",".join([
        "temperature_2m_max","temperature_2m_min","precipitation_sum",
        "windspeed_10m_max","windgusts_10m_max","shortwave_radiation_sum",
        "snowfall_sum"
    ])

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat(),
        "daily": daily_vars,
        "timezone": "Europe/Zurich"
    }
    resp = requests.get(ERA5_URL, params=params)
    if resp.status_code == 200:
        df = pd.DataFrame(resp.json()["daily"])
        # Simplified weathercode
        df["weathercode"] = df.apply(lambda row:
                                     "snow" if row.get("snowfall_sum",0)>0 else
                                     "rain" if row["precipitation_sum"]>5 else
                                     "showers/windy" if row["precipitation_sum"]>0 and row["windspeed_10m_max"]>10 else
                                     "sunny" if row["shortwave_radiation_sum"]>15000 else "cloudy",
                                     axis=1)
        return df
    else:
        print("Historical request failed:", resp.status_code)
        return None

def get_forecast_weather(n_days):
    """
    Fetch forecast daily weather for the next n_days.
    Converts official weathercode to simplified types like historical data.
    """
    FORECAST_URL = "https://api.open-meteo.com/v1/forecast"
    start_date = date.today()
    end_date = start_date + timedelta(days=n_days-1)

    daily_vars = [
        "temperature_2m_max","temperature_2m_min",
        "precipitation_sum","windspeed_10m_max",
        "weathercode","snowfall_sum"
    ]

    params = {
        "latitude": latitude,
        "longitude": longitude,
        "daily": ",".join(daily_vars),
        "start_date": start_date.isoformat(),
        "end_date": end_date.isoformat(),
        "timezone": "Europe/Zurich"
    }
    resp = requests.get(FORECAST_URL, params=params)
    if resp.status_code == 200:
        df = pd.DataFrame(resp.json()["daily"])

        # map official weathercode to simplified types
        def code_to_simple(wc, precip, wind, snow):
            if snow > 0:
                return "snow"
            elif precip > 5:
                return "rain"
            elif precip > 0 and wind > 10:
                return "showers/windy"
            elif wc in [0,1,2]:  # clear/sunny codes
                return "sunny"
            else:
                return "cloudy"

        df["weathercode"] = df.apply(lambda row: code_to_simple(
            row["weathercode"], row["precipitation_sum"], row["windspeed_10m_max"], row.get("snowfall_sum",0)
        ), axis=1)
        return df
    else:
        print("Forecast request failed:", resp.status_code)
        return None


In [ ]:
print("=== Historical Weather ===")
try:
    hist_df = get_historical_weather("2025-09-15", "2025-09-28")
    if hist_df is not None:
        print(hist_df)
except Exception as e:
    print("Historical weather fetch error:", e)

# ===============================
# Test forecast weather
# ===============================
print("\n=== Forecast Weather ===")
try:
    fc_df = get_forecast_weather(5)  # next 5 days
    if fc_df is not None:
        print(fc_df)
except Exception as e:
    print("Forecast weather fetch error:", e)

=== Historical Weather ===
          time  temperature_2m_max  temperature_2m_min  precipitation_sum  \
0   2025-09-15                23.9                16.1               22.2   
1   2025-09-16                18.2                11.7                0.0   
2   2025-09-17                19.7                 8.3                0.0   
3   2025-09-18                24.5                 9.9                0.0   
4   2025-09-19                26.2                 8.4                0.0   
5   2025-09-20                27.7                13.3                0.0   
6   2025-09-21                22.5                15.6               28.9   
7   2025-09-22                15.1                11.5               30.2   
8   2025-09-23                12.9                 9.8                2.5   
9   2025-09-24                13.1                 9.8                5.4   
10  2025-09-25                13.8                 8.0                1.1   
11  2025-09-26                13.6               

***Input 3: Yield Data (Optional but Recommended)***

In [ ]:
df = pd.read_csv("FAOSTAT_data.csv")

keywords = ["Wheat", "Barley", "Corn", "Soya", "Rapeseed", "Sunflower"]

mask = df["Item"].str.contains('|'.join(keywords), case=False, na=False)
df_filtered = df[mask]
columns_to_keep = ["Area", "Item", "Year", "Unit", "Value"]
df_filtered = df_filtered[columns_to_keep]
print(df_filtered)


             Area    Item  Year   Unit     Value
78    Switzerland  Barley  2018     ha   27897.0
79    Switzerland  Barley  2018  kg/ha    6486.3
80    Switzerland  Barley  2018      t  180948.0
81    Switzerland  Barley  2019     ha   27178.0
82    Switzerland  Barley  2019  kg/ha    6997.1
...           ...     ...   ...    ...       ...
1639  Switzerland   Wheat  2022  kg/ha    5465.7
1640  Switzerland   Wheat  2022      t  486667.0
1641  Switzerland   Wheat  2023     ha   87057.0
1642  Switzerland   Wheat  2023  kg/ha    5139.7
1643  Switzerland   Wheat  2023      t  447449.0

[128 rows x 5 columns]


***Input 4: Administrative or grid boundaries (crop location)***


In [ ]:
gdf = gpd.read_file("gadm41_CHE_3.shp")
point = Point(7.187919, 46.849259)
region = gdf[gdf.geometry.contains(point)]
print(region)

           GID_3 GID_0      COUNTRY    GID_1    NAME_1 NL_NAME_1      GID_2  \
904  CHE.7.7.4_1   CHE  Switzerland  CHE.7_1  Fribourg        NA  CHE.7.7_1   

    NAME_2 NL_NAME_2    NAME_3 VARNAME_3 NL_NAME_3   TYPE_3     ENGTYPE_3  \
904  Sense        NA  Düdingen        NA        NA  Commune  Municipality   

    CC_3 HASC_3                                           geometry  
904   NA     NA  POLYGON ((7.18216 46.80989, 7.17835 46.81351, ...  


<class 'pandas.core.frame.DataFrame'>
